<a href="https://colab.research.google.com/github/Amir-D-Shadow/A-study-of-pattern-between-the-academic-behaviour-of-students/blob/main/Development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from numba import cuda,float64,int64
import numpy as np
import math

In [2]:
@cuda.jit("float64[:,:,:],float64[:,:,:],float64[:,:,:,:],int64,int64,int64,int64")
def func(A,dZ,W,stride,Hlim,Wlim,Clim):

  """
  A -- n_H_prev + 2 * opadH, n_W_prev + 2*opadW,n_C_prev
  dZ -- n_H,n_W,n_C
  W -- fH,fW,n_C_prev,n_C
  """

  n_H = cuda.threadIdx.x + cuda.blockDim.x * cuda.blockIdx.x
  n_W = cuda.threadIdx.y + cuda.blockDim.y * cuda.blockIdx.y
  n_C = cuda.threadIdx.z + cuda.blockDim.z * cuda.blockIdx.z

  if (n_H < Hlim) and (n_W < Wlim) and (n_C < Clim):

    fH,fW,n_C_prev,n_C = W.shape

    for h in range(fH):

      for w in range(fW):

        for c in range(n_C_prev):

          IMG_H = n_H * stride + h
          IMG_W = n_W * stride + w

          A[IMG_H,IMG_W,c] = A[IMG_H,IMG_W,c] + W[h,w,c,n_C] * dZ[n_H,n_W,n_C]

In [6]:
opadH,opadW,stride = 2,2,1
m = 2
number_of_filters = 8

tmp = np.ones((m,1,1,3))
W = np.ones((2,2,3,number_of_filters))
dZ = np.ones((m,2,2,number_of_filters))
A = np.pad(tmp,((0,0),(opadH,opadH),(opadW,opadW),(0,0)),mode="constant",constant_values=(0,0))

m,n_H,n_W,n_C = dZ.shape 

threadsperblock = (2,2,8)

blockspergrid_H = int(math.ceil(n_H/threadsperblock[0]))
blockspergrid_W = int(math.ceil(n_W/threadsperblock[1]))
blockspergrid_C = int(math.ceil(n_C/threadsperblock[2]))

blockspergrid = (blockspergrid_H,blockspergrid_W,blockspergrid_C)
W_device = cuda.to_device(W)

for i in range(m):

  dZ_device = cuda.to_device(dZ[i,:,:,:].copy())
  A_device = cuda.to_device(A[i,:,:,:].copy())

  func[blockspergrid,threadsperblock](A_device,dZ_device,W_device,stride,n_H,n_W,n_C)

  A[i,:,:,:] = A_device.copy_to_host()


res = A[:,opadH:-opadH,opadW:-opadW,:]

print(res.shape)


(2, 1, 1, 3)
